In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
with open('../Datasets/mini_shakespeare.txt', 'r', encoding='utf8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))
print(f"first 100 characters: {text[:100]}")

length of dataset in characters:  1115394
first 100 characters: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"unique characters: ", ''.join(chars))
print(f"number of unique characters: {vocab_size}")

unique characters:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
number of unique characters: 65


In [8]:
# create a mapping from character to integer and a reverse mapping
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [11]:
data = torch.tensor(encode(text), dtype=torch.long)
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]
print(f"train data size: {train_data.size()}, validation data size: {val_data.size()}")

train data size: torch.Size([1003854]), validation data size: torch.Size([111540])


In [ ]:
block_size = 8